This project aims to analyze and predict the profitability of restaurant menu items using data-driven techniques and machine learning models. The primary goal is to identify key factors that influence the profitability of menu items and develop a neural network model to classify menu items into different profitability categories (High, Medium, Low).

The dataset includes information on various menu items, their categories, ingredients, and prices. The analysis involves data preprocessing, feature engineering, exploratory data analysis (EDA), model training, and evaluation. Key metrics such as accuracy, precision, recall, and F1 score are used to assess the performance of the model.

Through this project, we aim to provide valuable insights that can help restaurant owners and managers optimize their menu for better profitability.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import tensorflow as tf
from kerastuner.tuners import RandomSearch
import seaborn as sns


In [2]:
#Importing data

!pip install pandas
from google.colab import files

uploaded = files.upload()


Saving restaurant_menu_optimization_data.csv to restaurant_menu_optimization_data.csv


In [3]:

# Reading the CSV file and assigning it to the variable 'data'
data = pd.read_csv('restaurant_menu_optimization_data.csv')

# Display the first few rows of the dataframe
print(data.head())


  RestaurantID MenuCategory               MenuItem  \
0         R003    Beverages                   Soda   
1         R001   Appetizers  Spinach Artichoke Dip   
2         R003     Desserts    New York Cheesecake   
3         R003  Main Course        Chicken Alfredo   
4         R002  Main Course          Grilled Steak   

                                         Ingredients  Price Profitability  
0                                   ['confidential']   2.55           Low  
1       ['Tomatoes', 'Basil', 'Garlic', 'Olive Oil']  11.12        Medium  
2           ['Chocolate', 'Butter', 'Sugar', 'Eggs']  18.66          High  
3  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  29.55          High  
4  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  17.73        Medium  


In [4]:
# Check for missing values in each column
missing_values = data.isnull().sum()
print("Missing values in each column:")
print(missing_values)

# Check for any rows with missing values
rows_with_missing_values = data[data.isnull().any(axis=1)]
print("\nRows with missing values:")
print(rows_with_missing_values)


Missing values in each column:
RestaurantID     0
MenuCategory     0
MenuItem         0
Ingredients      0
Price            0
Profitability    0
dtype: int64

Rows with missing values:
Empty DataFrame
Columns: [RestaurantID, MenuCategory, MenuItem, Ingredients, Price, Profitability]
Index: []


In [5]:
# Check data types of each column
data_types = data.dtypes
print("\nData types of each column:")
print(data_types)



Data types of each column:
RestaurantID      object
MenuCategory      object
MenuItem          object
Ingredients       object
Price            float64
Profitability     object
dtype: object


In [6]:

# Replace 'confidential' with NaN in the 'Ingredients' column
data['Ingredients'] = data['Ingredients'].replace('confidential', np.nan)

# Create a new feature 'IngredientsCount'
data['IngredientsCount'] = data['Ingredients'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else np.nan)

# Impute missing values in 'IngredientsCount' with the mean count of non-confidential items
mean_ingredients_count = data['IngredientsCount'].mean()
data['IngredientsCount'].fillna(mean_ingredients_count, inplace=True)

# Display the first few rows to verify the imputation
print(data.head())


  RestaurantID MenuCategory               MenuItem  \
0         R003    Beverages                   Soda   
1         R001   Appetizers  Spinach Artichoke Dip   
2         R003     Desserts    New York Cheesecake   
3         R003  Main Course        Chicken Alfredo   
4         R002  Main Course          Grilled Steak   

                                         Ingredients  Price Profitability  \
0                                   ['confidential']   2.55           Low   
1       ['Tomatoes', 'Basil', 'Garlic', 'Olive Oil']  11.12        Medium   
2           ['Chocolate', 'Butter', 'Sugar', 'Eggs']  18.66          High   
3  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  29.55          High   
4  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  17.73        Medium   

   IngredientsCount  
0                 1  
1                 4  
2                 4  
3                 4  
4                 4  


In [7]:

# Create a new feature 'PriceCategory' based on the 'Price' column

# Define the bins for the price ranges
bins = [2.00, 10.00, 20.00, 29.80]
labels = ['Low', 'Medium', 'High']

# Create a new column 'PriceCategory' using pd.cut
data['PriceCategory'] = pd.cut(data['Price'], bins=bins, labels=labels, right=False)

# Display the first few rows to verify the new column
print(data.head())

# Summary statistics for 'Price' and 'IngredientsCount'
summary_stats = data[['Price', 'IngredientsCount']].describe()
print("\nSummary Statistics for Price and Ingredients Count:")
print(summary_stats)



  RestaurantID MenuCategory               MenuItem  \
0         R003    Beverages                   Soda   
1         R001   Appetizers  Spinach Artichoke Dip   
2         R003     Desserts    New York Cheesecake   
3         R003  Main Course        Chicken Alfredo   
4         R002  Main Course          Grilled Steak   

                                         Ingredients  Price Profitability  \
0                                   ['confidential']   2.55           Low   
1       ['Tomatoes', 'Basil', 'Garlic', 'Olive Oil']  11.12        Medium   
2           ['Chocolate', 'Butter', 'Sugar', 'Eggs']  18.66          High   
3  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  29.55          High   
4  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  17.73        Medium   

   IngredientsCount PriceCategory  
0                 1           Low  
1                 4        Medium  
2                 4        Medium  
3                 4          High  
4                 4        Mediu

In [8]:
# Label encoding for 'MenuCategory'
label_encoder_menu = LabelEncoder()
data['MenuCategoryEncoded'] = label_encoder_menu.fit_transform(data['MenuCategory'])


# Target encoding for 'Profitability'
profitability_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
data['Profitability_encoded'] = data['Profitability'].map(profitability_mapping)

# Define a mapping
ordinal_encoding = {'Low': 0, 'Medium': 1, 'High': 2}

# Map the categories to integers
data['PriceCategory_Encoded'] = data['PriceCategory'].map(ordinal_encoding)

print(data.head())

  RestaurantID MenuCategory               MenuItem  \
0         R003    Beverages                   Soda   
1         R001   Appetizers  Spinach Artichoke Dip   
2         R003     Desserts    New York Cheesecake   
3         R003  Main Course        Chicken Alfredo   
4         R002  Main Course          Grilled Steak   

                                         Ingredients  Price Profitability  \
0                                   ['confidential']   2.55           Low   
1       ['Tomatoes', 'Basil', 'Garlic', 'Olive Oil']  11.12        Medium   
2           ['Chocolate', 'Butter', 'Sugar', 'Eggs']  18.66          High   
3  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  29.55          High   
4  ['Chicken', 'Fettuccine', 'Alfredo Sauce', 'Pa...  17.73        Medium   

   IngredientsCount PriceCategory  MenuCategoryEncoded  Profitability_encoded  \
0                 1           Low                    1                      1   
1                 4        Medium                 

In [9]:
# Check data types of each column
data_types = data.dtypes
print("\nData types of each column:")
print(data_types)


Data types of each column:
RestaurantID               object
MenuCategory               object
MenuItem                   object
Ingredients                object
Price                     float64
Profitability              object
IngredientsCount            int64
PriceCategory            category
MenuCategoryEncoded         int64
Profitability_encoded       int64
PriceCategory_Encoded    category
dtype: object


In [10]:
# Check data types of each column
data_types = data.dtypes
print("\nData types of each column:")
print(data_types)


Data types of each column:
RestaurantID               object
MenuCategory               object
MenuItem                   object
Ingredients                object
Price                     float64
Profitability              object
IngredientsCount            int64
PriceCategory            category
MenuCategoryEncoded         int64
Profitability_encoded       int64
PriceCategory_Encoded    category
dtype: object


In [34]:

# Assuming 'data' is your DataFrame

# Convert categorical features to numerical using one-hot encoding
# Explicitly cast to float to ensure compatibility with TensorFlow
X = pd.get_dummies(data[['PriceCategory_Encoded', 'MenuItem', 'MenuCategoryEncoded', 'Ingredients']]).astype(float)
# Subtract 1 from 'Profitability_encoded' and cast to float before one-hot encoding
y = to_categorical(data['Profitability_encoded'].astype(float) - 1)




In [35]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2)

Epoch 1/50
20/20 - 2s - 88ms/step - accuracy: 0.4469 - loss: 1.0498 - val_accuracy: 0.6812 - val_loss: 0.9381
Epoch 2/50
20/20 - 0s - 10ms/step - accuracy: 0.5453 - loss: 0.9290 - val_accuracy: 0.6875 - val_loss: 0.8383
Epoch 3/50
20/20 - 0s - 7ms/step - accuracy: 0.6094 - loss: 0.8794 - val_accuracy: 0.6875 - val_loss: 0.7759
Epoch 4/50
20/20 - 0s - 6ms/step - accuracy: 0.6109 - loss: 0.8339 - val_accuracy: 0.6750 - val_loss: 0.7301
Epoch 5/50
20/20 - 0s - 7ms/step - accuracy: 0.6234 - loss: 0.7997 - val_accuracy: 0.6875 - val_loss: 0.7037
Epoch 6/50
20/20 - 0s - 7ms/step - accuracy: 0.6375 - loss: 0.7800 - val_accuracy: 0.7125 - val_loss: 0.6827
Epoch 7/50
20/20 - 0s - 7ms/step - accuracy: 0.6219 - loss: 0.7593 - val_accuracy: 0.7125 - val_loss: 0.6726
Epoch 8/50
20/20 - 0s - 15ms/step - accuracy: 0.6344 - loss: 0.7782 - val_accuracy: 0.7188 - val_loss: 0.6600
Epoch 9/50
20/20 - 0s - 14ms/step - accuracy: 0.6078 - loss: 0.7792 - val_accuracy: 0.7125 - val_loss: 0.6585
Epoch 10/50
20/

In [39]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6821 - loss: 0.7514  
Test Accuracy: 0.6700


In [41]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [42]:

# Define a function to build the model
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(Dense(units=hp_units, activation='relu', input_dim=X_train.shape[1]))

    # Tune the dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(Dropout(rate=hp_dropout))

    # Add more layers with tuning options
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout_' + str(i), min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='menu_profitability')

# Perform the search
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The optimal number of units in the first densely-connected layer is {best_hps.get('units')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)


Trial 10 Complete [00h 00m 11s]
val_accuracy: 0.71875

Best val_accuracy So Far: 0.7250000238418579
Total elapsed time: 00h 02m 29s

The optimal number of units in the first densely-connected layer is 160.
The optimal learning rate for the optimizer is 0.0004373927953369722.

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4423 - loss: 1.0377 - val_accuracy: 0.6438 - val_loss: 0.8360
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5721 - loss: 0.8895 - val_accuracy: 0.6438 - val_loss: 0.7316
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5972 - loss: 0.7909 - val_accuracy: 0.6750 - val_loss: 0.6719
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6711 - loss: 0.7254 - val_accuracy: 0.6812 - val_loss: 0.6577
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6781 - loss: 0.6924 - val_accuracy: 0.7063 - val_loss: 0.6343
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6443 - loss: 0.6803 - val_accuracy: 0.7125 - val_loss: 0.6293
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6685 - loss: 0.6730 - val_accuracy: 0.7063 - val_loss: 0.6267
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6547 - loss: 0.7100 - val_accuracy: 0.7125 - val_loss: 0.6242
Ep

In [43]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6798 - loss: 0.7414  
Test Accuracy: 0.6650


In [44]:

# Build a more complex neural network model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Output layer for 3 classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4684 - loss: 1.0533 - val_accuracy: 0.6438 - val_loss: 0.8633
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5638 - loss: 0.9036 - val_accuracy: 0.6562 - val_loss: 0.7942
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5524 - loss: 0.8966 - val_accuracy: 0.6687 - val_loss: 0.7451
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5359 - loss: 0.8667 - val_accuracy: 0.6875 - val_loss: 0.7064
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6071 - loss: 0.8189 - val_accuracy: 0.6812 - val_loss: 0.6784
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6087 - loss: 0.8145 - val_accuracy: 0.7063 - val_loss: 0.6608
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5834 - loss: 0.7981 - val_accuracy: 0.7125 - val_loss: 0.6502
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6676 - loss: 0.7593 - val_accuracy: 0.7125 - val_loss: 0

In [47]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6840 - loss: 0.7370 
Test Accuracy: 0.6650


In [46]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr])


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6689 - loss: 0.6885 - val_accuracy: 0.7125 - val_loss: 0.6141 - learning_rate: 0.0010
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6641 - loss: 0.7105 - val_accuracy: 0.7125 - val_loss: 0.6163 - learning_rate: 0.0010
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6416 - loss: 0.7059 - val_accuracy: 0.7125 - val_loss: 0.6184 - learning_rate: 0.0010
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6575 - loss: 0.6857 - val_accuracy: 0.7125 - val_loss: 0.6175 - learning_rate: 0.0010
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6667 - loss: 0.6803 - val_accuracy: 0.7125 - val_loss: 0.6224 - learning_rate: 0.0010
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6677 - loss: 0.7026 - val_accuracy: 0.7125 - val_loss: 0.6169 - learning_rate: 0.0010
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6579 - loss: 0.7379 - val_acc

In [49]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6840 - loss: 0.7370 
Test Accuracy: 0.6650


In [50]:
# Predict the test data
y_pred_prob = model.predict(X_test)
# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [51]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# If y_test is one-hot encoded, convert it to integer labels
y_test_labels = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')
f1 = f1_score(y_test_labels, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


Accuracy: 0.6650
Precision: 0.5988
Recall: 0.6650
F1 Score: 0.6141


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
